# Test plane extraction for a single point cloud

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import time
import open3d as o3d

import planeslam.general as general
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan
from planeslam.clustering import cluster_mesh_graph_search, mesh_cluster_pts
from planeslam.extraction import scan_from_clusters
import planeslam.io as io

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

Read Point Cloud from pcd file

In [ ]:
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)
P = PCs[8]

In [ ]:
# Convert points to ENU
P = general.NED_to_ENU(P)

# Downsample the points
P = general.downsample(P, factor=2, axis=0)

In [ ]:
# Plot the points
ax = general.plot_3D_setup(P)
ax.scatter3D(P[:,0], P[:,1], P[:,2], marker='.')  

Cluster the points

In [ ]:
# Create the mesh
start_time = time.time()
mesh = LidarMesh(P)
print("elapsed time: ", time.time() - start_time)

In [ ]:
start_time = time.time()
mesh.prune(edge_len_lim=10)
print("elapsed time: ", time.time() - start_time)

In [ ]:
# Cluster the mesh with graph search
start_time = time.time()
clusters, avg_normals = cluster_mesh_graph_search(mesh)
print("elapsed time: ", time.time() - start_time)

In [ ]:
#%lprun -f cluster_mesh_graph_search cluster_mesh_graph_search(mesh)

In [ ]:
# Plot mesh
ax = general.plot_3D_setup(P)

ax.plot_trisurf(P[:,0], P[:,1], P[:,2], triangles=mesh.DT.simplices)

In [ ]:
# Plot clusters
ax = general.plot_3D_setup(P)

cmap = cm.get_cmap('plasma')
cmap_idxs = int(len(cmap.colors) / len(clusters)) * np.arange(len(clusters))
colors = np.asarray(cmap.colors)[cmap_idxs,:]

for i, c in enumerate(clusters):
    cluster_pts = mesh_cluster_pts(mesh, c)
    ax.scatter3D(cluster_pts[:,0], cluster_pts[:,1], cluster_pts[:,2], color='C'+str(i), marker='.')  

    # Show normals
    c = np.mean(cluster_pts, axis=0)
    n = 10 * avg_normals[i]
    ax.quiver(c[0], c[1], c[2], n[0], n[1], n[2])

general.color_legend(ax, len(clusters))

Extract planes

In [ ]:
planes, vertices, faces = scan_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes, vertices, faces)

In [ ]:
ax = general.plot_3D_setup(P)
scan.plot(ax, show_normals=False)

In [ ]:
i = 1
c = clusters[i]
n = avg_normals[i][:,None]
cluster_pts = mesh_cluster_pts(mesh, c)  # Extract points from cluster

In [ ]:
cluster_pts

In [ ]:
from planeslam.extraction import bd_plane_from_pts
plane_pts = bd_plane_from_pts(cluster_pts, n)

In [ ]:
plane_pts

Plotly plotting

In [ ]:
import plotly.graph_objects as go
import numpy as np

layout = go.Layout(
             scene=dict(
                 aspectmode='data'
         ))
data=[go.Scatter3d(x=P[:,0], y=P[:,1], z=P[:,2],
                                   mode='markers',
                                   marker=dict(size=2))]
fig = go.Figure(data=data, layout=layout)
fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
mesh_data = go.Mesh3d(x=P[:,0], y=P[:,1], z=P[:,2], i=mesh.DT.simplices[:,0], j=mesh.DT.simplices[:,1], k=mesh.DT.simplices[:,2], flatshading=True)

#extract the lists of x, y, z coordinates of the triangle vertices and connect them by a line
Xe = []
Ye = []
Ze = []
for T in P[mesh.DT.simplices]:
    Xe.extend([T[k%3][0] for k in range(4)]+[ None])
    Ye.extend([T[k%3][1] for k in range(4)]+[ None])
    Ze.extend([T[k%3][2] for k in range(4)]+[ None])
       
#define the trace for triangle sides
lines = go.Scatter3d(
                   x=Xe,
                   y=Ye,
                   z=Ze,
                   mode='lines',
                   name='',
                   line=dict(color= 'rgb(70,70,70)', width=1))  

fig = go.Figure(data=[mesh_data, lines], layout=layout)
fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
cluster_idxs = np.zeros(len(P))
for i, c in enumerate(clusters):
    idxs = np.unique(mesh.DT.simplices[c,:]) 
    cluster_idxs[idxs] = i

In [ ]:
import plotly.express as px

fig = px.scatter_3d(P, x=0, y=1, z=2, color=cluster_idxs.astype(str))
fig.update_layout(width=1000, height=600, scene=dict(
                 aspectmode='data'
         ))
fig.update_traces(marker_size=2)
fig.show()

In [ ]:
def plotly_plane(p):
    mesh = go.Mesh3d(x=p.vertices[:,0], y=p.vertices[:,1], z=p.vertices[:,2], opacity=0.5)
    lines = go.Scatter3d(x=p.vertices[:,0], y=p.vertices[:,1], z=p.vertices[:,2], mode='lines', line=dict(color= 'rgb(70,70,70)', width=1))
    plane_data = [mesh, lines]
    return plane_data

In [ ]:
import plotly.graph_objects as go

data = []

for p in scan.planes:
    data += plotly_plane(p)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
data

In [ ]:
import plotly.graph_objects as go

i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]
j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]
k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]

triangles = np.vstack((i,j,k)).T

x = [0, 0, 1, 1, 0, 0, 1, 1]
y = [0, 1, 1, 0, 0, 1, 1, 0]
z = [0, 0, 0, 0, 1, 1, 1, 1]
vertices = np.vstack((x,y,z)).T
tri_points = vertices[triangles]

#extract the lists of x, y, z coordinates of the triangle vertices and connect them by a line
Xe = []
Ye = []
Ze = []
for T in tri_points:
    Xe.extend([T[k%3][0] for k in range(4)]+[ None])
    Ye.extend([T[k%3][1] for k in range(4)]+[ None])
    Ze.extend([T[k%3][2] for k in range(4)]+[ None])
       
#define the trace for triangle sides
lines = go.Scatter3d(
                   x=Xe,
                   y=Ye,
                   z=Ze,
                   mode='lines',
                   name='',
                   line=dict(color= 'rgb(70,70,70)', width=1))  

fig = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=[0, 0, 1, 1, 0, 0, 1, 1],
        y=[0, 1, 1, 0, 0, 1, 1, 0],
        z=[0, 0, 0, 0, 1, 1, 1, 1],
        colorbar_title='z',
        colorscale=[[0, 'gold'],
                    [0.5, 'mediumturquoise'],
                    [1, 'magenta']],
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity = np.linspace(0, 1, 8, endpoint=True),
        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        name='y',
        showscale=True
    )
, lines])

fig.show()

In [ ]:
tri_points